In [ ]:
import numpy as np
from itertools import islice
from scipy.stats import multivariate_normal

def performance(y, pred):
  m = np.zeros((3,3)) # confusion matrix
  for p in range(len(pred)):
    if pred[p]==1 and y[p]==1:
      m[0,0]+=1
    if pred[p]==2 and y[p]==2:
      m[1,1]+=1
    if pred[p]==3 and y[p]==3:
      m[2,2]+=1
    if pred[p]==1 and y[p]==2:
      m[1,0]+=1
    if pred[p]==1 and y[p]==3:
      m[2,0]+=1
    if pred[p]==2 and y[p]==1:
      m[0,1]+=1
    if pred[p]==2 and y[p]==3:
      m[2,1]+=1
    if pred[p]==3 and y[p]==1:
      m[0,2]+=1
    if pred[p]==3 and y[p]==2:
      m[1,2]+=1
  ind_accuracy = [m[0,0]/np.sum(m[0,:]), m[1,1]/np.sum(m[1,:]), m[2,2]/np.sum(m[2,:])]
  accuracy = (m[0,0]+m[1,1]+m[2,2])/np.sum(m)
  return ind_accuracy, accuracy

In [ ]:
# getting data from file
data = []
filename = '/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 1/data_q6_q7.txt'
with open(filename) as file:
  lines = list(islice(file,None,None,None))
  for line in lines:
    entries = line.split('\t')
    while '' in entries: entries.remove('')
    y = int(entries[-1].replace('\n',''))
    entries.pop()
    v = list(map(float,entries))
    v.append(y)
    data.append(v)

data = np.concatenate((np.ones((np.shape(data)[0],1)),np.array(data)), axis=1) #appending ones
np.random.shuffle(data) # shuffling data
x = np.array(data[:,:np.shape(data)[1]-1])
y = np.array(data[:,-1])

# train-test split
m = len(y)
trainp = int(np.floor(0.7*m)) # 70% training

y_train = y[0:trainp]
y_test = y[trainp:]

x_train = x[0:trainp,:]
x_test = x[trainp:,:]

# normalizing
qq = np.amax(np.abs(x_train), axis=0)
x_train = x_train/qq
x_test = x_test/qq

# not normalizing output data for classification

m_train = len(y_train)
m_test = len(y_test)

In [ ]:
# Maximum Likelihood Classifier

# data for likelihood
x_class1_train = x_train[np.where(y_train==1)]
x_class2_train = x_train[np.where(y_train==2)]
x_class3_train = x_train[np.where(y_train==3)]

# likelihood modeled by multivariate gaussian
pdfXgiven1 = multivariate_normal.pdf(x_test, np.mean(x_class1_train, axis=0), np.cov(x_class1_train, rowvar=False), allow_singular=True)
pdfXgiven2 = multivariate_normal.pdf(x_test, np.mean(x_class2_train, axis=0), np.cov(x_class2_train, rowvar=False), allow_singular=True)
pdfXgiven3 = multivariate_normal.pdf(x_test, np.mean(x_class3_train, axis=0), np.cov(x_class3_train, rowvar=False), allow_singular=True)

# prediction
pred = []
for i in range(m_test):
  pdf_vector = [pdfXgiven1[i],pdfXgiven2[i],pdfXgiven3[i]]
  pred.append(np.argmax(pdf_vector)+1)

# performance measures
ind_accuracy, accuracy = performance(y_test, pred)

print("accuracy of class 1 = {}".format(ind_accuracy[0]))
print("accuracy of class 2 = {}".format(ind_accuracy[1]))
print("accuracy of class 3 = {}".format(ind_accuracy[2]))
print("overall accuracy of classifier = {}".format(accuracy))

accuracy of class 1 = 0.95
accuracy of class 2 = 1.0
accuracy of class 3 = 1.0
overall accuracy of classifier = 0.9841269841269841
